<a href="https://colab.research.google.com/github/ishneha1/AI_CourseWork/blob/main/AI_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries**



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (classification_report,
                            confusion_matrix,
                            accuracy_score)
from sklearn.preprocessing import LabelEncoder

# **Loading Dataset and Understanding data**


In [2]:
data = pd.read_csv('/content/sample_data/Dhaka Obesity.csv')
# Show first 5 rows to understand the data structure
print(data.head())

   Gender  Age  Height (m)  Weight (kg) Family history of overweight  \
0    Male   29        1.65        101.0                          Yes   
1  Female   25        1.65         53.0                           No   
2    Male   23        1.70         70.0                           No   
3    Male   22        1.68        112.0                          Yes   
4    Male   19        1.75         67.0                           No   

  High caloric food consumption Vegetable consumption frequency  \
0                           Yes                       Sometimes   
1                            No                          Always   
2                            No                          Always   
3                           Yes                       Sometimes   
4                           Yes                          Always   

  Daily main meals frequency Between-meal food consumption frequency Smoking  \
0                      Three                              Frequently      No   
1   

# **Data Cleaning**

In [4]:
# Check for missing values
print(data.isnull().sum())
# Remove duplicate rows if any
initial_count = len(data)
data.drop_duplicates(inplace=True)
final_count = len(data)
# Check for impossible values in numerical columns
print(data.describe())

Gender                                     0
Age                                        0
Height (m)                                 0
Weight (kg)                                0
Family history of overweight               0
High caloric food consumption              0
Vegetable consumption frequency            0
Daily main meals frequency                 0
Between-meal food consumption frequency    0
Smoking                                    0
Alcohol intake                             0
Daily water intake                         0
Monitor calories                           0
Physical exercise                          0
Daily device usage duration                0
Mode of transportation                     0
Obesity level                              0
dtype: int64
               Age   Height (m)  Weight (kg)
count  2157.000000  2157.000000  2157.000000
mean     26.766806     1.627144    74.021604
std       8.446704     0.101490    20.787374
min      15.000000     1.220000    35.0000